In [27]:
# Import Dependencies
import pymongo
import requests
import pandas as pd
import time
from bs4 import BeautifulSoup
from splinter import Browser
from urllib.parse import urljoin
from pymongo import MongoClient
from sqlalchemy import create_engine

In [75]:
browser = Browser('chrome')
url = 'https://twitchtracker.com/'
browser.visit(url)

In [76]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [77]:
def scrape_all(url):

    page_number = 0
    all_games = []

    for x in range(1,11):
        page_number = page_number +1
        browser.visit(f'https://twitchtracker.com/games?page={x}')
        html = browser.html
        soup = BeautifulSoup(html, 'html.parser')

        current_games = top_games(page_number)
        all_games = all_games + current_games
    return all_games
    

In [78]:
def top_games(page_number):
    game_list = []
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    all_div = soup.body.findAll('div', class_ = "ranked-item")

    for div in all_div:
        game = game_info(div)
        game_list.append(game) 
    return game_list

In [79]:
def game_info(html_element):
    game_data = {}
    streamer_data = []
    game_data['Current Rank'] = html_element.find('div', class_ = "ri-position hidden-xxs").text
    game_data['Game Title'] = html_element.find('div', class_ = "ri-name").text
    game_data['Live Viewers'] = html_element.find('div', class_ = "ri-value").text
    
    link = html_element.a['href']
    game_streamers_url = urljoin(url, link)
    browser.click_link_by_href(link)
#     game_link.click()
#     game_streamers_url.click()
    streamer_data['Streamers'] = top_streamers(game_streamers_url)    
    browser.back()


    
    return game_data

In [83]:
def top_streamers(url):
    streamer_list = {}
#     browser.visit(url)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    tables = soup.find_all("tbody")
    
    top_list = []
#     bottom_list = []
    for top in tables[3].find_all("td"):
        top_list.append(top.text.strip())
#     for bottom in tables[4].find_all("td"):
#         bottom_list.append(bottom.text.strip())
    
    top_list_names = top_list[::2]
#     bottom_list_names = bottom_list[::2]
#     streamer_names_list = top_list_names + bottom_list_names
    
    top_list_views = top_list[1::2]
#     bottom_list_views = bottom_list[1::2]
#     streamer_views_list = top_list_views + bottom_list_views
    
    streamer_list['Streamers'] = top_list_names
    streamer_list['Live Views'] = top_list_views
    
        
    return streamer_list

In [84]:
scrape_all(url)

TypeError: list indices must be integers or slices, not str

In [85]:
top_streamers('https://twitchtracker.com/games/509658')

{'Streamers': ['HasanAbi',
  'IlloJuan',
  'ZanoXVII',
  'MarkitoNavaja',
  'CashApp',
  'Homyatol',
  'orslok',
  'chefstrobel',
  'Tumblurr',
  'Im_Dontai'],
 'Live Views': ['46,172',
  '17,453',
  '17,177',
  '14,957',
  '11,363',
  '11,225',
  '10,669',
  '10,208',
  '9,311',
  '8,515']}